In [2]:
import os, sys, shutil
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib as plt
from PIL import Image
from matplotlib.lines import Line2D
import matplotlib as mpl
import math
import matplotlib.image as mpimg
import random
# os.chdir("..")

In [13]:
#Experiment column key:
# 1: Experiment 1, mindeye vs second sight
# 2: Experiment 2, second sight two way identification
# 3: Experiment 3, mental imagery two way identification
df_exp = pd.DataFrame(columns=["experiment", "stim1", "stim2", "stim3", "sample", "subject", "target_on_left", "iter_count", "is_low", "method", "catch_trial", "rep", "mode"])
i=0
random_count = 0
for subj in [1,2,5,7]: #1,2,5,7
    for sample in tqdm(range(982)):
        #count how many iterations the sample searched for
        iter_files = os.listdir(f"../output/mindeye_extension_v6/subject{subj}/{sample}/")
        iter_count = sum(1 for file in iter_files if "iter_" in file and file.endswith(".png"))
        
        #Experiment 1, mindeye vs second sight
        order = random.randrange(2)
        sample_names = [f"{sample}_subject{subj}_mindeye", f"{sample}_subject{subj}_secondsight"]
        
        left_sample = sample_names.pop(order)
        right_sample = sample_names.pop()
        gt_sample = f"{sample}_ground_truth"
        df_exp.loc[i] = {"experiment" : 1, "stim1" : gt_sample, "stim2" : left_sample, "stim3" : right_sample, "sample" : sample, "subject" : subj, 
                         "target_on_left" : order == 1, "iter_count" : iter_count, "is_low" : False, "method" : None, "catch_trial" : None, "rep" : 0, "mode" : "nsd_vision"}
        i+=1
        
        #Experiment 2, second sight two way identification
        order = random.randrange(2)
        shuffled_idx = [i for i in range(982)]
        random.shuffle(shuffled_idx)
        random_sample = shuffled_idx[sample]
        sample_names = [f"{random_sample}_subject{subj}_secondsight", f"{sample}_subject{subj}_secondsight"]
        
        left_sample = sample_names.pop(order)
        right_sample = sample_names.pop()
        gt_sample = f"{sample}_ground_truth"
        df_exp.loc[i] = {"experiment" : 2, "stim1" : gt_sample, "stim2" : left_sample, "stim3" : right_sample, "sample" : sample, "subject" : subj, 
                         "target_on_left" : order == 1, "iter_count" : iter_count, "is_low" : False, "method" : None, "catch_trial" : None, "rep" : 0, "mode" : "nsd_vision"}
        i+=1
    print(i)
    
    #Experiment 3, mental imagery two way identification
    random_count = {"mindeye" : 0, "braindiffuser" : 0, "tagaki" : 0, "secondsight" : 0}
    random_low_count = {"mindeye" : 0, "braindiffuser" : 0, "tagaki" : 0, "secondsight" : 0}
    for mode in ["vision", "imagery"]:
        for sample in range(12):
            gt_sample = f"mi_{sample}_ground_truth"
            for method in ["mindeye", "braindiffuser", "tagaki", "secondsight"]: 
                for rep in range(5):
                    order = random.randrange(2)
                    if method in ["mindeye", "secondsight"]:
                        shuffled_idx = [i for i in range(982)]
                        random.shuffle(shuffled_idx)
                        random_sample = shuffled_idx[sample]
                        sample_names = [f"{random_sample}_subject{subj}_{method}", f"mi_{sample}_subject{subj}_{method}_{mode}_{rep}"]
                    else:
                        sample_names = [f"{random_count[method]}_subject{subj}_{method}_random", f"mi_{sample}_subject{subj}_{method}_{mode}_{rep}"]
                        random_count[method] +=1
                    
                    left_sample = sample_names.pop(order)
                    right_sample = sample_names.pop()
                    #count how many iterations the sample searched for
                    if method == "secondsight":
                        iter_files = os.listdir(f"../output/ss_mi_{mode}/subject{subj}/{sample}/")
                        iter_count = sum(1 for file in iter_files if "iter_" in file and file.endswith(".png"))
                    else:
                        iter_count=None
                    
                    df_exp.loc[i] = {"experiment" : 3, "stim1" : gt_sample, "stim2" : left_sample, "stim3" : right_sample, "sample" : sample, "subject" : subj, 
                                    "target_on_left" : order == 1, "iter_count" : iter_count, "is_low" : False, "method" : method, "catch_trial" : None, "rep" : rep, "mode" : mode}
                    i+=1
                if method != "secondsight":
                    order = random.randrange(2)
                    sample_names = [f"{random_low_count[method]}_subject{subj}_{method}_low_random", f"mi_{sample}_subject{subj}_{method}_{mode}_low"]
                    random_low_count[method] +=1
                    left_sample = sample_names.pop(order)
                    right_sample = sample_names.pop()
                    df_exp.loc[i] = {"experiment" : 3, "stim1" : gt_sample, "stim2" : left_sample, "stim3" : right_sample, "sample" : sample, "subject" : subj, 
                                    "target_on_left" : order == 1, "iter_count" : None, "is_low" : True, "method" : method, "catch_trial" : None, "rep" : rep, "mode" : mode}
                    i+=1                 
    print(i)
print(df_exp)
df_exp = df_exp.sample(frac=1)
print(df_exp)

  0%|          | 0/982 [00:00<?, ?it/s]

100%|██████████| 982/982 [00:13<00:00, 72.10it/s] 


1964
2516


100%|██████████| 982/982 [00:20<00:00, 47.87it/s]


4480
5032


100%|██████████| 982/982 [00:21<00:00, 45.79it/s]


6996
7548


100%|██████████| 982/982 [00:21<00:00, 45.61it/s]


9512
10064
       experiment               stim1                                 stim2  \
0               1      0_ground_truth                    0_subject1_mindeye   
1               2      0_ground_truth              129_subject1_secondsight   
2               1      1_ground_truth                1_subject1_secondsight   
3               2      1_ground_truth                1_subject1_secondsight   
4               1      2_ground_truth                2_subject1_secondsight   
...           ...                 ...                                   ...   
10059           3  mi_11_ground_truth              507_subject7_secondsight   
10060           3  mi_11_ground_truth  mi_11_subject7_secondsight_imagery_1   
10061           3  mi_11_ground_truth  mi_11_subject7_secondsight_imagery_2   
10062           3  mi_11_ground_truth              237_subject7_secondsight   
10063           3  mi_11_ground_truth              628_subject7_secondsight   

                                      st

In [14]:
# Check if all images are present in final stimuli folder
count_not_found = 0
stim_path = "/home/naxos2-raid25/kneel027/home/kneel027/Second-Sight/experiments/final_stimuli/"
for index, row in df_exp.iterrows():
    if not os.path.exists(f"{stim_path}{row['stim1']}.png"):
        print(f"{row['stim1']}.png")
        count_not_found += 1
    if not os.path.exists(f"{stim_path}{row['stim2']}.png"):
        print(f"{row['stim2']}.png")
        count_not_found += 1
    if not os.path.exists(f"{stim_path}{row['stim3']}.png"):
        print(f"{row['stim3']}.png")
        count_not_found += 1
print(count_not_found)

0


In [15]:
#Add participant ID column
pIDs = []
for i in range(len(df_exp)):
    pIDs.append(i // 105)
df_exp.insert(0, "pID", pIDs)
print(len(df_exp[(df_exp['pID'] == 0)]))
#Add catch trials within each pID section
for pID in range(max(pIDs)):
    df_pid = df_exp[(df_exp['experiment'] == 2) & (df_exp['pID'] == pID)]
    
    # Ground truth catch trials
    gt_catch_trials = df_pid.sample(n=5)
    gt_catch_trials['catch_trial'] = "ground_truth"
    for index, row in gt_catch_trials.iterrows():
        
        order = random.randrange(2)
        ground_truth = row['stim1']
        stims = [row['stim2'], ground_truth]
        
        gt_catch_trials.at[index, 'stim2'] = stims.pop(order)
        gt_catch_trials.at[index, 'stim3'] = stims.pop()
        # Target on left here means the ground truth repeat is on the left
        gt_catch_trials.at[index, 'target_on_left'] = (order == 1)
        
    # repeated trial catch trials
    repeat_catch_trials_rep1 = df_pid.sample(n=5)
    repeat_catch_trials_rep1['catch_trial'] = "repeat"
    repeat_catch_trials_rep2 = repeat_catch_trials_rep1.copy()
    repeat_catch_trials_rep1['rep'] = 1
    repeat_catch_trials_rep2['rep'] = 2
    df_exp = pd.concat([df_exp, gt_catch_trials, repeat_catch_trials_rep1, repeat_catch_trials_rep2])
    
df_exp = df_exp.sample(frac=1).sort_values(by='pID', kind='mergesort')
print(df_exp)
print(len(df_exp[(df_exp['pID'] == 0)]))

105
      pID  experiment              stim1                            stim2  \
5291    0           2   129_ground_truth         129_subject5_secondsight   
1810    0           1   905_ground_truth             905_subject1_mindeye   
3482    0           1   483_ground_truth         483_subject2_secondsight   
7079    0           3  mi_3_ground_truth        17_subject5_tagaki_random   
7415    0           3  mi_6_ground_truth   18_subject5_mindeye_low_random   
...   ...         ...                ...                              ...   
1829   95           2   914_ground_truth         859_subject1_secondsight   
4771   95           3  mi_0_ground_truth   mi_0_subject2_tagaki_imagery_3   
4063   95           2   773_ground_truth         773_subject2_secondsight   
374    95           1   187_ground_truth         187_subject1_secondsight   
9903   95           3  mi_5_ground_truth  mi_5_subject7_mindeye_imagery_0   

                                  stim3  sample  subject  target_on_lef

In [16]:
version = 4
df_exp.to_csv(f'../experiments/dataframes/experiment_v{version}.csv', index=False)
df_exp_tsv = df_exp[['pID', 'stim1', 'stim2', 'stim3']].copy()
df_exp_tsv.to_csv(f"../experiments/dataframes/experiment_v{version}_meadow_trials.tsv", sep="\t", index=False, header=False) 

# THE FOLLOWING CELLS ARE FOR PROCESSING RESPONSES

In [25]:
response_path = "../experiments/responses/"
dataframe_path = "../experiments/dataframes/"
df_experiment = pd.read_csv(dataframe_path + "experiment_v1.csv")
df_responses = pd.read_csv(response_path + "Quinn_test1.csv")
df_responses = df_responses[df_responses['participation'] == "poetic-mongoose"]

In [26]:
df_responses.head()
df_trial = pd.DataFrame(columns=["experiment", "stim1", "stim2", "stim3", "sample", "subject", "target_on_left", "iter_count", "is_low", "method", "catch_trial", "rep", "mode", "picked_left"])
df_experiment['picked_left'] = None
for index, row in tqdm(df_responses.iterrows()):
    if row['label'] == row['stim2_id']:
        picked_left = True
    elif row['label'] == row['stim3_id']:
        picked_left = False
    else:
        print("Error")
        break
    # df_experiment[(df_experiment['ground_truth'] == row['stim1_name']) & (df_experiment['stim1'] == row['stim2_name']) & (df_experiment['stim2'] == row['stim3_name'])]['picked_left'] = picked_left
    df_trial.loc[index] = df_experiment[(df_experiment['ground_truth'] == row['stim1_name']) & (df_experiment['stim1'] == row['stim2_name']) & (df_experiment['stim2'] == row['stim3_name'])].iloc[0]
    df_trial.loc[index, 'picked_left'] = picked_left
df_trial["picked_target"] = df_trial["picked_left"] == df_trial["target_on_left"]
print(df_trial)

0it [00:00, ?it/s]/tmp/ipykernel_112238/630132413.py:13: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df_trial.loc[index] = df_experiment[(df_experiment['ground_truth'] == row['stim1_name']) & (df_experiment['stim1'] == row['stim2_name']) & (df_experiment['stim2'] == row['stim3_name'])].iloc[0]
/tmp/ipykernel_112238/630132413.py:13: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df_trial.loc[index] = df_experiment[(df_experiment['ground_truth'] == row['stim1_name']) & (df_experiment['stim1'] == row['stim2_name']) & (df_experiment['stim2'] == row['stim3_name'])].iloc[0]
/tmp/ipykernel_112238/630132413.py:13: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only

    experiment                     stim1                     stim2  stim3  \
0            2  886_subject2_secondsight  536_subject2_secondsight    NaN   
1            1  207_subject7_secondsight      207_subject7_mindeye    NaN   
2            1    3_subject2_secondsight            3_ground_truth    NaN   
3            2  503_subject1_secondsight  565_subject1_secondsight    NaN   
4            2  787_subject7_secondsight  960_subject7_secondsight    NaN   
..         ...                       ...                       ...    ...   
115          1   93_subject7_secondsight       93_subject7_mindeye    NaN   
116          2  839_subject1_secondsight  451_subject1_secondsight    NaN   
117          2  372_subject1_secondsight   24_subject1_secondsight    NaN   
118          1      562_subject7_mindeye  562_subject7_secondsight    NaN   
119          1  896_subject5_secondsight      896_subject5_mindeye    NaN   

    sample subject target_on_left iter_count is_low method   catch_trial re

In [27]:

df_trial_exp1 = df_trial[(df_trial['experiment'] == 1)]
# df_trial_exp1 = df_trial[(df_trial['experiment'] == 3) & (df_trial['mode'] == "imagery")]
# df_trial_exp1 = df_trial[(df_trial['experiment'] == 3) & (df_trial['method'] == "mindeye")]
print(len(df_trial_exp1))
print(len(df_trial_exp1[df_trial_exp1["picked_target"]]) / len(df_trial_exp1))

59
0.5423728813559322
